In [1]:
# 主任务：利用爬虫收集新闻信息，进行人物观点分析
# 本脚本任务：爬取信息并保存到数据库中
import re

import pandas
import requests
from bs4 import BeautifulSoup
from urllib import request
import chardet

In [2]:
# 利用百度搜索引擎按关键词搜索热点新闻
#keyword = input("请输入关键词：")
keyword = "李现"
url_baidu = "http://www.baidu.com/s?tn=news&word=" + keyword
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

In [6]:
# 定义函数用于提取百度搜索页面的新闻网址和下一页网址

def get_url(url):
    
     # 提取搜索页面网址    
    search_result = requests.get(url, headers=headers).text
    search_result = search_result.replace("\n","")   # 删除换行符，因为换行符不好匹配

    # 首先定位搜索结果所在的区间，避免引入大量垃圾网址
    results_re = r'<div class="result" id="\d+">(.*)</div>'
    results = re.compile(results_re).findall(search_result)

    # 在区间内搜索网址
    url_news_re = r'<a href="(https?://\S*)"'
    url_news = re.compile(url_news_re).findall(str(results))
    #print(url_news)
    
    # 提取下一页按钮网址
    url_next_page_re = '<a href="(\S*)"\ class="n">下一页&gt;</a>'
    url_next_page_0 = re.compile(url_next_page_re).findall(str(results))
    url_next_page = 'http://www.baidu.com' + url_next_page_0[0]
    print(url_next_page)
    return url_news, url_next_page
    

In [7]:
# 打开搜索结果中的网址，爬取新闻信息
def extract_text_by_bs(doc_text):
    """提取html页面的所有文本信息。 """
    title, text = '', ''
    soup = BeautifulSoup(doc_text, 'lxml')
    try:
        for script in soup(["script", "style"]):
            script.extract()
    except Exception as error:
        print('error')
        pass
    else:
        try:
            # get text
            title = soup.title.string
        except Exception as error:
            print('error')
            pass
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
    return title, text

In [10]:
# 设置访问页数
max_number_of_page = 5 

# 用于保存每个网页的新闻信息
news = []

# 第一次，给定地址
url_news, url_next_page = get_url(url_baidu)

# 第二次以后，都用下一页地址
for page_number in range(max_number_of_page):
    url_news, url_next_page = get_url(url_next_page)

    # 开始遍历url_news中的每个网址，提取网页内容 
    for url in url_news:
        
        # 临时网址无效，不需要登录
        if 'cache' in url:
            continue
        
        # 提取网页信息，并判断其编码方式
        news_page = requests.get(url, headers=headers)
        
        # 如果不知道是什么编码，则放弃这条新闻，不是好办法，先凑合着用
        try:
            response = request.urlopen(url)
            html = response.read()
            charset = chardet.detect(html)     #对该html进行编码的获取
        except:
            continue
        
        # 如果不是utf_8编码，则放弃这条新闻，不是好办法，先凑合着用
        if charset['encoding'] != 'utf-8':
            continue
        news_page.encoding = "utf-8"

        # 登录不上的网址，不继续进行解析
        if "503 Service Temporarily Unavailable" in news_page.text:
            continue
        
        
        title, text = extract_text_by_bs(news_page.text)
        print(text[:100])
        news.append(text)

    print(len(news))

http://www.baidu.com/s?tn=news&word=%E6%9D%8E%E7%8E%B0&x_bfe_rqs=03E80&x_bfe_tjscore=0.491112&tngroupname=organic_news&rsv_dl=news_b_pn&pn=10
http://www.baidu.com/s?tn=news&word=%E6%9D%8E%E7%8E%B0&x_bfe_rqs=03E80&x_bfe_tjscore=0.135461&tngroupname=organic_news&rsv_dl=news_b_pn&pn=20
新晋男友李现身陷被包养传闻，被同性圈追捧？前女友身份让人意外_钟楚曦 搜狐首页 新闻 体育 汽车 房产 旅游 教育 时尚 科技 财经 娱乐 更多 母婴 健康 历史 军事 美食 文化 星座 专题 游戏 
腾讯新闻
李现，有什么好喜欢的？百度首页登录个人中心帐号设置意见反馈退出李现，有什么好喜欢的？闲人电影发布时间：08-0121:50优质原创作者这个夏天的七月都被一个人承包了。这个人叫做李现，他的粉丝都是他的现
杨紫自曝在合作过的男演员中，李现最帅，与邓伦最有CP感_娱乐频道_凤凰网 首页 资讯 视频 直播 财经 娱乐 体育 时尚 汽车 房产 科技 读书 游戏 文化 历史 军事 旅游 佛教 更多 国学 数码 
「新片新剧」李现2018-2019年有哪些电影电视剧作品|拍了什么戏_新戏名单 - 漫漫看影视剧 电视剧 明星库 电视节目表 综艺节目 电影 电视剧 首页 2019电视剧 内地 网络剧 电视剧上映时间
百度用户服务中心-首页 百度帐号被封禁 如何举报网站 投诉侵权信息 百度帐号被盗 帐号申诉未通过 贴吧贴子被删 删除/更新快照 登录 注册 服务支持 账号安全 密码及密保工具管理 身份认证及账户关联 
6
http://www.baidu.com/s?tn=news&word=%E6%9D%8E%E7%8E%B0&x_bfe_rqs=03E80&x_bfe_tjscore=0.128319&tngroupname=organic_news&rsv_dl=news_b_pn&pn=30
女粉都喊着李现老公，可看到李妈妈颜值后，网友：这谁敢嫁啊_手机网易网 女粉都喊着李现老公，可看到李妈妈颜值后，网友：这谁敢嫁

In [11]:
# 将新闻信息保存到文件中
import pickle
pickle.dump(news, open("news.pkl","wb"))

In [12]:
print(news)

['新晋男友李现身陷被包养传闻，被同性圈追捧？前女友身份让人意外_钟楚曦 搜狐首页 新闻 体育 汽车 房产 旅游 教育 时尚 科技 财经 娱乐 更多 母婴 健康 历史 军事 美食 文化 星座 专题 游戏 搞笑 动漫 宠物 社会先知道 由内容质量、互动评论、分享传播等多维度分值决定，勋章级别越高( )，代表其在平台内的综合表现越好。 文章 总阅读 新晋男友李现身陷被包养传闻，被同性圈追捧？前女友身份让人意外 2019-07-18 13:03 来源:社会先知道 原标题：新晋男友李现身陷被包养传闻，被同性圈追捧？前女友身份让人意外 暖心爱恋剧《亲爱的，热爱的》开播以来，韩商言的扮演者李现也凭借其自然生动的表演给观众留下了深刻的印象，成为大家心目中公认的 " 现男友 "。 近日，有人爆料称李现曾经被富婆圈养过，还放出了该富婆提到李现的朋友圈。这位富婆晒出了一张李现的写真，她称自己Ｓ过几个小鲜肉，还称李现是一个 "goodkisser"。由于之前坊间就曾有李现被圈养的传闻流出，所以这则爆料一出，有不少瓜众选择相信。 不过，没过多久这位富婆就在微博辟谣，说朋友圈截图是别人ｐ的，自己老公看到会气死。虽然这位富婆辟谣了和李现没有什么关系，但这位富婆的私生活还是比较 " 丰富 " 的，还在微博上发过自己近乎ｌｔ的照片。所以，李现幸好没和她扯上什么关系，不然被扒出来就凉了。李现这段时间的势头还是很猛的，热度和话题度都很高，杨天真目前也在各种造势帮李现炒作，有八百营销号出动，想必接下来李现的热搜至少是不用发愁了。 李现是壹心娱乐的艺人，在张若昀主演的《法医秦明》中崭露头角，26 岁时凭借网剧《河神》走红 那么既然小哥哥现在火了，那么大家当然不会放过这个了解他的机会了，万能的网友们挖出了他的前女友，居然是钟楚曦！不仅被扒出来而且还上了热搜。 \u200b 钟楚曦大家都听说过吧？华谊兄弟力捧小花，有着强大的背景，走霸气御姐风格。 2 年前主演了冯小刚的电影《芳华》走红，虽然是女二但是号称 " 力压 " 女主角苗苗而引起了大家的注意，在今年因 " 聂小倩 " 的扮相又获得一波热度。对于钟楚曦，其实大家被她吸引的是颜值和身材，不曾想她居然还是李现的前女友？ 网友发现李现和钟楚曦在 2014 年就互动过，那时候两人都是小透明。有网友还曝料，李现会从北京坐火车去上海看钟楚曦，其实两人从外形看还很配，

In [ ]:
# 简单处理新闻信息

# 将新闻信息保存到数据库中